## Grid World Value Iteration

<img src="img/bombs and gold numbers.png" style="width: 320px;" align="left" caption="Figure 1"/>

**Actions available:** The agent has four possible actions in each grid square. These are *west*, *north*, *south*, and *east*. If the direction of movement is blocked by a wall (for example, if the agent executes action south in grid square 1), the agent remains in the same grid square. 

**Collecting gold:** On its first arrival at a grid square that contains gold (from a neighbouring grid square), the agent collects the gold. Note that, in order to collect the gold, the agent needs to transition into the grid square (containing the gold) from a different grid square.

**Hitting the bomb:** On arrival at a grid square that contains a bomb (from a neighbouring grid square), the agent activates the bomb. 

**Terminal states:** The game terminates when **all** gold is collected or when the bomb is activated. In Exercises 1 and 2, you can define terminal states to be grid squares 18 and 23. In Exercise 3, you will need to define terminal state(s) differently.

**Reward function:** $-1$ for each navigation action (including when the action results in hitting the wall), an additional $+10$ for collecting each piece of gold, and an additional $-10$ for activating the bomb. 

Discounting is not used (i.e. $\gamma=1$)


In [1]:
# environment
import numpy as np

class Gridworld:
    def __init__(self):
        self.num_rows = 5
        self.num_cols = 5
        self.num_cells = self.num_cols * self.num_rows
        
        # Choose starting position of the agent randomly among the first 5 cells
        self.agent_position = np.random.randint(0, 5)
        
        # Choose position of the gold and bomb
        self.bomb_positions = np.array([18])
        self.gold_positions = np.array([23])
       
        # Specify rewards
        self.rewards = np.zeros(self.num_cells)
        self.rewards[self.bomb_positions] = -10
        self.rewards[self.gold_positions] = 10
        
        # Specify available actions
        self.actions = ["UP", "RIGHT", "DOWN", "LEFT"]
        self.num_actions = len(self.actions)
    
    def setLocation(self, position):
        self.agent_position = position
    
    def makeStep(self, action_index): 
        action = self.actions[action_index]

        # Determine new position and check whether the agent hits a wall.
        old_position = self.agent_position
        new_position = self.agent_position
        if action == "DOWN":
            candidate_position = old_position + self.num_cols
            if candidate_position < self.num_cells:
                new_position = candidate_position
        elif action == "RIGHT":
            candidate_position = old_position + 1
            if candidate_position % self.num_cols > 0:  # The %-operator denotes "modulo"-division.
                new_position = candidate_position
        elif action == "UP":
            candidate_position = old_position - self.num_cols
            if candidate_position >= 0:
                new_position = candidate_position
        elif action == "LEFT":  # "LEFT"
            candidate_position = old_position - 1
            if candidate_position % self.num_cols < self.num_cols - 1:
                new_position = candidate_position
        
        # Calculate reward
        reward = self.rewards[new_position]
        reward -= 1
        return reward, new_position

This first task uses the Bellman equation for value iteration however assumes that an action is guaranteed to successfully take place. It outputs the optimal policy for the environment.

In [2]:
env = Gridworld()
policy = np.empty(25, dtype = np.unicode_)
v = np.zeros(25)
theta = 1e-10
valueAction = [0, 0, 0, 0]
sigma = 1
difference = theta + 1
actions = ['n', 'e', 's', 'w']

# loop thru 
while difference > theta:
    difference = 0
    for state in range (25):
        # check if state is terminal, if terminal skip
        if state == 18 or state == 23:
            continue
        currentStateValue = v[state]
        # move agent to current state in environment
        env.setLocation(state)
        # for each action in the state, find valueAction
        for action in range (4):
            reward, position = env.makeStep(action)
            valueAction[action] = reward + (sigma*v[position])
        # find max valueAction
        v[state] = max(valueAction[0], valueAction[1], valueAction[2], valueAction[3])
        # policy improvement
        policy[state] = actions[valueAction.index(v[state])]
        difference = max(difference, abs(currentStateValue - v[state]))
print(v.reshape((5, 5)))
print(policy.reshape((5, 5)))

[[3. 4. 5. 4. 5.]
 [4. 5. 6. 5. 6.]
 [5. 6. 7. 6. 7.]
 [6. 7. 8. 0. 8.]
 [7. 8. 9. 0. 9.]]
[['e' 'e' 's' 'e' 's']
 ['e' 'e' 's' 'e' 's']
 ['e' 'e' 's' 'e' 's']
 ['e' 'e' 's' '' 's']
 ['e' 'e' 'e' '' 'w']]


This second tasks continues using value iteration however the agent now fails its action 20% of the time.

In [3]:
env = Gridworld()
policy = np.empty(25, dtype = np.unicode_)
v = np.zeros(25)
theta = 1e-10
valueAction = [0, 0, 0, 0]
sigma = 1
difference = theta + 1
actions = ['n', 'e', 's', 'w']

# loop thru 
while difference > theta:
    difference = 0
    for state in range (25):
        # check if state is terminal, if terminal skip
        if state == 18 or state == 23:
            continue
        currentStateValue = v[state]
        # move agent to current state in environment
        env.setLocation(state)
        # for each action in the state, find valueAction
        for action in range (4):
            reward, position = env.makeStep(action)
            #  80% chance of successful action, 20% chance that action fails
            valueAction[action] = round((0.8 * (reward + (sigma*v[position]))) + (0.2 * (-1 + (sigma * v[state]))), 2)
        # find max valueAction
        v[state] = max(valueAction[0], valueAction[1], valueAction[2], valueAction[3])
        # policy improvement
        policy[state] = actions[valueAction.index(v[state])]
        difference = max(difference, abs(currentStateValue - v[state]))


print(v.reshape((5, 5)))
print(policy.reshape((5, 5)))

[[1.25 2.5  3.75 2.5  3.75]
 [2.5  3.75 5.   3.75 5.  ]
 [3.75 5.   6.25 5.   6.25]
 [5.   6.25 7.5  0.   7.5 ]
 [6.25 7.5  8.75 0.   8.75]]
[['e' 'e' 's' 'e' 's']
 ['e' 'e' 's' 'e' 's']
 ['e' 'e' 's' 'e' 's']
 ['e' 'e' 's' '' 's']
 ['e' 'e' 'e' '' 'w']]


<img src="img/bomb and two gold.png" style="width: 300px;" align="left" caption="Figure 1"/>
The final task uses a similar stochastic environment as the previous, however now includes two gold objects. A terminal state is only reached when both gold pieces are collected, or the bomb is activated.

In [4]:
# environment

class Ex3Gridworld:
    def __init__(self):
        self.num_rows = 5
        self.num_cols = 5
        self.num_cells = self.num_cols * self.num_rows
        
        # Choose starting position of the agent randomly among the first 5 cells
        self.agent_position = np.random.randint(0, 5)
        
        # Choose position of the gold and bomb
        self.bomb_positions = np.array([18])
        self.gold_positions = np.array([23, 12])
       
        # Specify rewards
        self.rewards = np.zeros(self.num_cells)
        self.rewards[self.bomb_positions] = -10
        self.rewards[self.gold_positions] = 10
        
        # Specify available actions
        self.actions = ["UP", "RIGHT", "DOWN", "LEFT"]
        self.num_actions = len(self.actions)
    
    def setLocation(self, position):
        self.agent_position = position
    
    def makeStep(self, action_index): 
        action = self.actions[action_index]

        # Determine new position and check whether the agent hits a wall.
        old_position = self.agent_position
        new_position = self.agent_position
        if action == "DOWN":
            candidate_position = old_position + self.num_cols
            if candidate_position < self.num_cells:
                new_position = candidate_position
        elif action == "RIGHT":
            candidate_position = old_position + 1
            if candidate_position % self.num_cols > 0:  # The %-operator denotes "modulo"-division.
                new_position = candidate_position
        elif action == "UP":
            candidate_position = old_position - self.num_cols
            if candidate_position >= 0:
                new_position = candidate_position
        elif action == "LEFT":  # "LEFT"
            candidate_position = old_position - 1
            if candidate_position % self.num_cols < self.num_cols - 1:
                new_position = candidate_position
        
        # Calculate reward
        reward = self.rewards[new_position]
        reward -= 1
        return reward, new_position

In [ ]:
env = Ex3Gridworld()
policy = np.empty(25, dtype = np.unicode_)
v = np.zeros(25)
theta = 1e-10
valueAction = [0, 0, 0, 0]
sigma = 1
difference = theta + 1
actions = ['n', 'e', 's', 'w']

# loop thru 
while difference > theta:
    difference = 0
    for state in range (25):
        # check if state is terminal, if terminal skip
        if state == 18:
            continue
        # if both state 12 and 23 visited then continue ???
        # TO DO
        currentStateValue = v[state]
        # move agent to current state in environment
        env.setLocation(state)
        # for each action in the state, find valueAction
        for action in range (4):
            reward, position = env.makeStep(action)
            #  80% chance of successful action, 20% chance that action fails
            valueAction[action] = round((0.8 * (reward + (sigma*v[position]))) + (0.2 * (-1 + (sigma * v[state]))), 2)
        # find max valueAction
        v[state] = max(valueAction[0], valueAction[1], valueAction[2], valueAction[3])
        # policy improvement
        policy[state] = actions[valueAction.index(v[state])]
        difference = max(difference, abs(currentStateValue - v[state]))


print(v.reshape((5, 5)))
print(policy.reshape((5, 5)))